<a href="https://colab.research.google.com/github/acmeyer/idea-generator/blob/main/gpt_idea_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Idea Generator

This program is for helping you to generate ideas by leveraging GPT (through OpenAI's API). The basics for how it works is that you provide it a prompt about what kinds of ideas you're looking for (examples are also provided below) and GPT will generate a list of ideas for you. You can then use this list however you want (for example, it could spur new ideas for you or you can filter and sort them by your own personal ranking, etc).

## How it works

The easiest thing to do is go to **Runtime** > **Run all**. This will run all the below cells for you, without you having to run them one by one. You will need to input some things in order for the program to complete, but you will be prompted for this information when needed.

Alternatively, you can run each cell one-by-one by clicking the run button next to the cell (looks like a "play" button).

If you want to make any changes to this program or save a copy for yourself, go to **File** > **Save a copy in Drive**.

## Setup

This section contains the code necessary to run the program. As part of the setup, you will need to enter your OpenAI API Key.

### Packages

This section contains the necessary packages to run the program. You can ignore this section unless you want to add new packages.

In [ ]:
# Install required packages
!pip install openai --quiet
!pip install tenacity --quiet

In [ ]:
# Import necessary packages
import openai
import pandas as pd
import os
from io import StringIO
from tenacity import retry, stop_after_attempt, wait_random_exponential, retry_if_not_exception_type

### OpenAI Configuration

This section configures settings for OpenAI. You will need to enter your API Key here but you can also change the model and temperature settings here.

In [ ]:
openai_api_key = input('Enter your OpenAI API Key: ')
openai.api_key = openai_api_key
MODEL = "gpt-4" # Change this to gpt-3.5-turbo if you want faster and cheaper (but generally worse) results
TEMPERATURE = 0.7 # Change this value if you want more or less creativity (but also hallucinations) 0 is low, 1 is high

### Functions

These functions provide the main logic to this program. If you want to make changes to the logic, you can do that here. Otherwise, you don't need to pay attention to this section.

In [ ]:
@retry(wait=wait_random_exponential(min=1, max=20), stop=stop_after_attempt(6), retry=retry_if_not_exception_type(openai.InvalidRequestError))
def get_completion(messages):
    response = openai.ChatCompletion.create(
        model=MODEL,
        messages=messages,
        temperature=TEMPERATURE,
    )
    return response['choices'][0]['message']['content']

In [ ]:
def get_new_ideas(system_prompt: str, num_new_ideas: int = 10, existing_ideas: list[str] = []):
    user_content = f"Please generate {num_new_ideas} new ideas with each idea as a row in a csv with 'name' and 'description' as headers. Each description should be a paragraph of 40-80 words."
    if len(existing_ideas) > 0:
        user_content += "\n\nHere are the existing ideas. Do not repeat any of these when generating new ones:\n" + "\n- ".join(existing_ideas)

    messages=[
        {"role": "system", "content": f"{system_prompt} Your output should be in a CSV format with one idea per row and with the headings 'name' and 'description'."},
        {"role": "user", "content": user_content},
    ]
    return get_completion(messages)

In [ ]:
def generate_ideas(system_prompt: str, n: int = 10):
    output_df = pd.DataFrame({
        'name': [],
        'description': [],
    })
    ideas: list[str] = []
    # split idea generation into batches of 10
    batches = int(n / 10)
    # for each batch, generate 10 ideas
    for i in range(batches):
        new_ideas = get_new_ideas(system_prompt, 10, ideas)
        # parse csv output into list of ideas
        csv_file = StringIO(new_ideas)
        df = pd.read_csv(csv_file)
        # Merge with output_df
        output_df = pd.concat([output_df, df])
        # join name and description into one string
        df['idea'] = df['name'] + ': ' + df['description']
        # append to list of ideas
        ideas.extend(df['idea'].tolist())

    return output_df

## Output

This section contains all the necessary code for generating the output from GPT and storing it. You will need to provide a system prompt for what you want to tell the model to do (i.e. what types of ideas you want it to generate and what contraints it should consider) and how many ideas you would like it to generate (10 is the default).

For the final output, the default program set up is to print the table of results for you. If you would also like to store the output data as a CSV file in your drive, uncomment the "Google Drive" section code below.

### System Prompt

In this section, you need to provide instructions to the model for what you want it to do. In order to help, there are a few examples of prompts that you can use or modify for your needs. You can also come up with your own if you would like.

In [ ]:
# Example system prompts

SAAS_BUSINESS_PROMPT = """You are a creative entrepreneur looking to generate new product ideas.
The product will target other technology companies in the United States.
It should be a software product, not a physical good or a service.
I'd like for it to be a SaaS product, or software as a service.
Customers should be business customers, so the product should be a B2B style business product.
It should be a product that you can charge customers somewhere between $20 - $1,000 per month.
The ideas are just ideas. The product need not yet exist, nor may it necessarily be clearly feasible."""

MARKETER_CONTENT_PROMPT = """You are a creative marketer for a technology startup.
Your job is to generate a series of ideas for creative marketing campaigns that our company can run.
Our company sells email marketing software to ecommerce businesses.
The campaign ideas should be innovative and off the beaten path."""

In [ ]:
system_prompt = input("What kinds of ideas do you want generated (aka the system prompt)? ")

### Number of Ideas

Here, you need to provide the number of ideas you want generated for you. Keep in mind that the more ideas you generate, the longer it will take and the more money it will cost. In addition, It may error out if you ask for too many ideas at once, given the model's context window limitations. A good range is somewhere between 10-30 ideas.

In [ ]:
num_of_ideas = input("How many ideas would you like generated? ")

### Ideas

Running the below cells will print out your output into a table you can view.

In [ ]:
output_df = generate_ideas(system_prompt, int(num_of_ideas))
output_df

### Google Drive

If you would like to store the output in your Google Drive, you will need to uncomment the below code cell and run it yourself. Note: Google will ask you permission to access your Google Drive in order to save this information there.

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')
# filename = input("What would you like the file to be saved as (you don't need to include the .csv extension)? ")
# output_df.to_csv(f'/content/drive/MyDrive/{filename}.csv', index=False)